# Case 2: Dynamic Group Size (Fixed $b_q$)

In [6]:
import sys
sys.path.append("/mnt/rds/VipinRDS/VipinRDS/users/mxh1029/projects/compression/spectral-kv")

from kvq.group_pattern import group_pattern


if __name__ == "__main__":

    models = [
        "meta-llama/Llama-3.2-1B-Instruct",
        "meta-llama/Llama-3.2-3B-Instruct",
        "meta-llama/Llama-3.1-8B-Instruct",
        "meta-llama/Llama-3.3-70B-Instruct",
        "Qwen/Qwen3-0.6B",
        "Qwen/Qwen3-4B",
        "Qwen/Qwen3-8B",
        "Qwen/Qwen3-32B",
    ]

    
    group_fixed_budget = [32, 64, 128]

    scores = [0, 1]  # 0 for frobenius_norm, 1 for spectral_norm



    for model in models:
        for budget in group_fixed_budget:
            for score in scores:
                kv_bits = group_pattern(
                    model_name_or_path=model,
                    group_size_budget=budget,
                    group_sizes=[16, 32, 64, 128, 256],
                    
                    score=score,
                )
                print(f"Model: {model}, Budget: {budget}, Score: {score}")
                print(kv_bits)
                print("\n")

Model: meta-llama/Llama-3.2-1B-Instruct, Budget: 32, Score: 0
{'g_k': [16, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32], 'g_v': [64, 64, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]}


Model: meta-llama/Llama-3.2-1B-Instruct, Budget: 32, Score: 1
{'g_k': [16, 16, 32, 16, 32, 16, 16, 32, 32, 16, 32, 16, 32, 32, 32, 32], 'g_v': [64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 32, 32]}


Model: meta-llama/Llama-3.2-1B-Instruct, Budget: 64, Score: 0
{'g_k': [32, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64], 'g_v': [128, 128, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64]}


Model: meta-llama/Llama-3.2-1B-Instruct, Budget: 64, Score: 1
{'g_k': [32, 32, 64, 32, 64, 32, 32, 64, 64, 32, 64, 32, 64, 64, 64, 64], 'g_v': [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 64, 64]}


Model: meta-llama/Llama-3.2-1B-Instruct, Budget: 128, Score: 0
{'g_k': [64, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,

In [8]:
TOL: float = 1e-6  # Tolerance for floating point comparisons


models = [
    "meta-llama/Llama-3.2-1B-Instruct",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.1-8B-Instruct",
    "meta-llama/Llama-3.3-70B-Instruct",
    "Qwen/Qwen3-0.6B",
    "Qwen/Qwen3-4B",
    "Qwen/Qwen3-8B",
    "Qwen/Qwen3-32B",
]

group_fixed_budget = 64

for model in models:

    kv_groups = group_pattern(
        model_name_or_path=model,
        group_size_budget=group_fixed_budget,
        group_sizes=[16, 32, 64, 128],
        score=1,
    )

    print("w_k group sizes:", kv_groups["g_k"])
    print("w_v group sizes:", kv_groups["g_v"])

    n_layers = len(kv_groups["g_k"])

    budget_using_fixed_group_size = 2 * n_layers / group_fixed_budget

    budget_using_dynamic_group_size = sum(1 / g for g in kv_groups["g_k"]) + sum(
        1 / g for g in kv_groups["g_v"]
    )

    print(
        f"Model: {model} | "
        f"Budget using fixed group size: {budget_using_fixed_group_size:.6f} | "
        f"Budget using dynamic group size: {budget_using_dynamic_group_size:.6f}"
    )

    assert (
        abs(budget_using_fixed_group_size - budget_using_dynamic_group_size) < TOL
    ), f"Budget mismatch for model {model!r}."
    print("\n")


w_k group sizes: [32, 32, 64, 32, 64, 32, 32, 64, 64, 32, 64, 32, 64, 64, 64, 64]
w_v group sizes: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 64, 64]
Model: meta-llama/Llama-3.2-1B-Instruct | Budget using fixed group size: 0.500000 | Budget using dynamic group size: 0.500000


w_k group sizes: [32, 32, 64, 32, 32, 64, 32, 32, 32, 64, 64, 32, 64, 64, 32, 32, 32, 64, 64, 32, 64, 64, 64, 64, 64, 64, 64, 32]
w_v group sizes: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 64, 64, 128]
Model: meta-llama/Llama-3.2-3B-Instruct | Budget using fixed group size: 0.875000 | Budget using dynamic group size: 0.875000


w_k group sizes: [32, 32, 64, 64, 32, 32, 32, 32, 64, 64, 64, 64, 32, 64, 64, 32, 32, 32, 32, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 32, 64, 32]
w_v group sizes: [64, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 12